# <i>Please Note this Notebook was completed within an ArcGIS Pro Notebook environment with access to Arcpy</i>

## Part 1: Data loading/converting for each individual image location
1. Load Data
    * DEM
    * GPS csv file (results from "MetaData-GPS.ipynb")
2. Convert csv file to coordinates
3. Convert GPS coordinates (a feature layer) to shp
4. Create individual shps for each image point
    * Extract relevant values for names (names for new shapefiles)
    * Create 6 copies of the GPS shp (containing all data) and delete all rows expt 1 point (then save shp)

## Part 2: Extracting elevation values (maxes) in a 360 view around image points. 

1. Use Viewshed() to calculate viewshed rasters for each image
2. Viewshed raster to Polygon
3. Create a minimum bounding circle on vectorized viewshed raster
4. Generate points on perimeter of circle (min bounding region)
5. Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile.
6. Use vectorized viewshed raster to mask DEM (resulting in a viewshed shaped DEM with real elevation values)
7. Extract the cell values from the clipped viewshed DEM with the Polylines. 
    * For each line store the largest DEM value and save values in a table

## Part 3: Calculate Distance from image/anchor pt to each Max Elevation 

1. Spatial Join between vectorized clipped DEM from viewshed and the polylines generated from image to min bounding pts
    * retain polygon (pixel) geometry
    * make sure it retains only MAX elevation polygon per line
2. Clip or divide the Polylines with the geom returned from previous step. 
    * retain only lines that intersect with image/anchor point
3. Measure the line length (Use "Calculate Geometry Attributes" tool)
4. Export a csv table that contains the "degree", "elevation" (m), and "length" (km) for each polyline.

In [6]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *
from pathlib import Path


def get_image_coords(path_to_image):
    """
    Given the shapefile path (of an image location), extract the point's coordinates.
    Note: this function assumes the provided shapefile is 1 point, thus it only returns 1 point value,
    if shapefile has many pts, funtion will return last pt

    Returns an arcpy Point
    """
    with arcpy.da.SearchCursor(path_to_image, ["*"]) as cursor:
        for row in cursor:
            cp = arcpy.Point(row[1][0], row[1][1])
    return cp


def creating_polylines(filepath, anchor_point, connection_points, percentage_increment):
    """
    Creates lines from each coordinate (image location) to the points plotted on the minimum bounding circle
    created around the viewshed.

    Inputs:
        filepath: insert desired name for each polyline file created with entire filepath
                  (recommend saving to .gdb)
        anchor_point: filepath to shapefile for each respective image's coordinates
        connection_points: filepath to the points created on the minimum bounding circle
        percentage_increment: dictates the frequency/interval of the pts plotted around the minimum bounding circle.
                              The samller the number the more points. Must be a value between 0 and 100.
                              (makes sure this 'percentage_increment' = 'Percentage' in "GeneratePointsAlongLines()")

    Returns:
        a new shapefile made of polylines labeled by a "Degree" column
    """
    # matching the spatial refernce of the DEM file
    sr = arcpy.SpatialReference(4269)
    arcpy.management.CreateFeatureclass(Output_Filepath, filepath, "POLYLINE", spatial_reference=sr)
    # ADDing a column that designates degree (I believe arc designates N as 0 and rotates clockwise)
    arcpy.management.AddField(
        filepath,
        "Degree",
        "DOUBLE",
        None,
        None,
        None,
        "Degree",
        "NULLABLE",
        "NON_REQUIRED",
        "")

    percentage = 0

    # Creating a line for each point on perimeter to corresponding image
    # looping through perimeter points
    with arcpy.da.SearchCursor(connection_points, ["*"]) as cursor:
        for row in cursor:
            list_points = arcpy.Array([])
            # calculating the degree sequentially clockwise
            # (make sure percentage = GeneratePointsAlongLines() percentage)
            degree = (percentage / 100) * 360

            # declaring start (image location) & end (pts on circle) points for each line
            cp1 = get_image_coords(anchor_point)  # pulling the same image point
            cp2 = arcpy.Point(row[1][0], row[1][1])
            list_points.append(cp2)
            list_points.append(cp1)

            # assigning spatial reference
            spatial_reference = arcpy.SpatialReference(4326)
            polyline = arcpy.Polyline(list_points, spatial_reference)

            # inserting new line to new feature class
            with arcpy.da.InsertCursor(filepath, ["SHAPE@", "Degree"]) as icur:
                icur.insertRow([polyline, degree])
                del icur

            # updating percentage for each line
            percentage += percentage_increment


def Identifying_Max_elevations_from_Poly_Pixels(line_geom, poly_pixel_geom):
    """
    Inputs:
        line_geom: the shapefile for the polylines that radiate from and around image location
        poly_pixel_geom: the pixels and their respective elevation value that are visible
                        (based on viewshed) for each image location (should be a polygon geom)

    Returns:
        a dictionary that identifes the DEM pixels that contain the MAX elevation for each polyline.

        dictionary format: max_dict[Degree] = [pixel_ID, coords (lat,long), Max_Elevation]
                       Assumes that the "poly_pixel_geom" feature always has the same columns in a certain 
                       order (will extract wrong values if columns are not the same for all geoms)
    """
    max_dict = {}
    counter = 0
    # For each line:
    with arcpy.da.SearchCursor(line_geom, ["Degree"]) as cursor:
        for line in cursor:
            ID = line[0]  # make sure to grab "Degree" so as to match unique values later
            # iterate over corresponding pixels that match each line (through 'ID' column)
            with arcpy.da.SearchCursor(poly_pixel_geom, ["*"]) as cur:
                Max_ = 0
                obj_id = 0
                coords = 0
                for pixel in cur:
                    counter += 1
                    # make sure these indeces match correct columns!!
                    if pixel[6] == ID:
                        if Max_ < pixel[5]:
                            Max_ = pixel[5]
                            obj_id = pixel[3]  # target_ID
                            coords = pixel[1]
                        max_dict[ID] = [obj_id, coords, Max_]
    print(
        "Done Creating Max Dict. Iterated",
        counter,
        "times (lines x pixels in each viewshed)"
        )

    return max_dict
    # Fucntion works BUT I bet there is a better/faster way...


def Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict, poly_pixel_geom):
    """
    This function deletes the poylgons from the geometry (vectorized pixels) that have not been identified 
    as the pixels with max elevation for each polyline.
    Inputs:
        max_dict: resulting dict from 'Identifying_Max_elevations_from_Poly_Pixels()'
        poly_pixel_geom: the pixels and their respective elevation value that are visible
                        (based on viewshed) for each image location (should be a polygon geom)

    Returns:
        a subset/edited version of the provided "poly_pixel_geom"
        (will only work if the proper inputs have already been made)
    """
    # grabbing the OBJECTID's and using these values to delete non-max associated polygons
    Obj_ID_list = []
    for item in max_dict.values():
        # there are pixel polygons IDs that are duplicated in this list, only need to add them once
        # (no need for repeating IDs)
        if item[0] not in Obj_ID_list:
            Obj_ID_list.append(item[0])

    safety = 0
    # deleteting polygons whose target_IDs are NOT the max_dict max_dict)
    with arcpy.da.UpdateCursor(poly_pixel_geom, ["*"]) as cursor:
        for row in cursor:
            if row[3] not in Obj_ID_list and safety < 4:
                cursor.deleteRow()

    # still need to delete more pixel polygons if there do not contain the max elevetion per line.
    # (slower than the above Search Cursor)
    with arcpy.da.UpdateCursor(poly_pixel_geom, ["*"]) as cursor:
        for row in cursor:
            # checking if the Target Id and the elevation are within the max_dict, if not, delete the row
            if row[3] not in max_dict[row[6]] and row[5] not in max_dict[row[6]]:
                cursor.deleteRow()


def Splitting_polylines_by_Max_Pixel_geom(
    poly_pixel_max_geom, 
    intersection_geom, 
    polylines_geom, 
    Split_line_geom, 
    max_dict
    ):
    """
    Clip lines where they intersect MAX elavtion pixels within 360 degree view of image location.

    Inputs:
        poly_pixel_max_geom: polygon shapefile with elevation attribute for visisble DEM pixels (based on viewshed)
        intersection_geom: name for points created based on the intersection of 'polylines_geom' and 'poly_pixel_max_geom'
        polylines_geom: the lines created from "creating_polylines()' function
        Split_line_geom: name for resulting polyline file split by intersection_geom
        max_dict: resulting dict from 'Identifying_Max_elevations_from_Poly_Pixels()'
    Returns:
        primary file: line geom (shapefile) for lines that intersect the original image location and end at each respective
        Max Elevation pixel 360 degree around each image location
        secondary file: point geom (shapefile) for the points that mark the intersection of 'polylines_geom' and 
        'poly_pixel_max_geom'
    """
    # rename "Target_FID" col inside the vectorized pixel geom features that have been spatialy joined with lines.
    # make a new field then copy the old field values into the new one
    arcpy.management.CalculateField(
        poly_pixel_max_geom,
        "PIXEL_ID",
        "!TARGET_FID!",
        "PYTHON3",
        "",
        "LONG",
        "NO_ENFORCE_DOMAINS"
        )
    # delete unnecessary columns
    arcpy.management.DeleteField(poly_pixel_max_geom, "TARGET_FID", "DELETE_FIELDS")

    # creating points that delinate the intersection of each line with each max polygon
    # (creates points whenever each line intersect a polygon)
    arcpy.analysis.Intersect(
        [polylines_geom, poly_pixel_max_geom],
        intersection_geom,
        "NO_FID",
        None,
        "POINT"
        )

    # need to keep only one point per line that marks where each line intersects with its max elevation pixel/polygon
    safety = 0
    number_of_matches = 0
    with arcpy.da.UpdateCursor(
        intersection_geom, ["gridcode", "Degree", "PIXEL_ID"]
        ) as cursor:
        for row in cursor:
            degree = row[1]
            # if pixel_id are equal and elevation values are equal keep
            if max_dict[degree][0] == row[2] and max_dict[degree][2] == row[0]:
                number_of_matches += 1
            # else delete row
            else:
                cursor.deleteRow()

    print("Number of matches found:", number_of_matches)

    # Split lines by points:
    arcpy.management.SplitLineAtPoint(
        polylines_geom, intersection_geom, Split_line_geom, None
        )

    # delete polyline with ORG_SEG = 1 (delete lines that do not intersect with anchor point):
    safety = 0
    with arcpy.da.UpdateCursor(Split_line_geom, ["ORIG_SEQ"]) as cursor:
        for row in cursor:
            if row[0] == 1:
                cursor.deleteRow()

    # Successfully clipped lines!!!

## Part 1:
### Creating Variables for Project
Adjust following cell to reflect current location for required data (DEM and MetaData csv) and desired default workspace for resulting files.

Recommendation: 
* set "Output_Filepath" to current project's .gdb (all shapefile porducts are saved to this directory) 
and 
* "out_path" to a non .gdb environment (all csv products are saved to this directory).

In [2]:
# adjust to accurate location of files
GPS_csv_Filepath = r"C:\Users\runac\Downloads\Fall_2022\Programming\project_idea\JupNotebooks\GPS_metadata.csv"
DEM_Filepath = r"D:\Fall_2022\Programming\Project\Data\30DEM\USGS_1_n49w114_20210607.tif"
Output_Filepath = "D:\Fall_2022\Programming\Project\Modified_Viewsheds\Modified_Viewsheds.gdb"

#saving resulting files to a NON .gdb directory
out_path = r"D:\Fall_2022\Programming\Project\Modified_Viewsheds"
# shp = List_shp[0] # (or any image generated in Part 1)

### Converting Data:
Converting the provided Metadata csv (coordinates for each image location) to individual shapefiles (so that each image is contained in seperate shapefiles).  

In [3]:
############################ Loading DEM and GPS data ############################
# load relevant DEM needed for the viewshed
# DEM1 = Raster(DEM_Filepath)

# pre-defining spatial refernce
spatial_ref = arcpy.SpatialReference(4326)

# convert the csv table to points (run once)
arcpy.management.XYTableToPoint(
    GPS_csv_Filepath,
    r"GPS_metadata_XYTableToPoint",
    "decimal_Long",
    "decimal_Lat",
    "GPSAltitude",
    spatial_ref
    )

# converting GPS data to a shp
arcpy.conversion.FeatureClassToShapefile("GPS_metadata_XYTableToPoint", Output_Filepath)

# creating a list that stores the name of each image file (List_shp can used later to create 360 ridgelines)
List_shp = []
List_names = []
with arcpy.da.UpdateCursor("GPS_metadata_XYTableToPoint", ["Field1"]) as cursor:
    for row in cursor:
        name = row[0].split(".")
        List_shp.append(f"{name[0]}.shp")
        List_names.append(row[0])

# copying featureclass and converting to individual shpfiles for each point
data = Output_Filepath + "\GPS_metadata_XYTableToPoint"
count = 0
for name in List_shp:
    # copying original file
    arcpy.management.CopyFeatures(data, name)
    # creating shpfiles with 1 point by deleting all features excepts for 1 row
    with arcpy.da.UpdateCursor(
        name, ["Field1", "decimal_La", "decimal_Lo", "GPSAltit_1"]
    ) as cursor:
        for row in cursor:
            if row[0] != List_names[count]:
                cursor.deleteRow()
    count += 1
    print("Created", name)

Created image1.shp
Created image2.shp
Created image3.shp
Created image4.shp
Created image5.shp
Created image8.shp


## Part 2 and Part 3:
 Chose one file to run the following function with (took 30 mins to execute). My previous version executed the code faster, but this code has been reformatted to execute image at a time (instead of all 6 simultaneously)
 
 All resulting files generated within the "Export_CSV_for_360_Ridgeline()" function are saved (by default to current project .gdb) while the csv is saved to a user specified location (recommend a directory that is not a .gdb)

In [4]:
def Export_CSV_for_360_Ridgeline(image_pt, DEM, out_file_name, out_path):
    """
    Input:
        image_pt: name (or location) of image shapefile
                  must be a point shapefile with 1 point
        DEM: a raster object of relevant DEM file
        out_file_name: desired name of returned csv (please do not include .csv in name) 
        out_path: desired location for saving the csv (please use '\' in path)
    Returns:
        saves a csv file to your desired 'out_path' with the specified 'out_file_name' as the name
        csv will contain 3 critical columns: 'Degree', 'Elevation' in meters , and 'Distance' in KM
    """
    ###################### VIEWSHED ######################
    # Execute Viewshed
    outViewshed = Viewshed(DEM, image_pt, 1, "FLAT_EARTH", 0.13)
    # Save the output 
    outViewshed.save("outViewshed")
    print("saving outViewshed")

    ###################### Viewshed raster to Polygon ######################
    # converting rasters to polygons
    arcpy.conversion.RasterToPolygon("outViewshed", 
                                     "Poly_outView", 
                                     "SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

    # deleting polygons with gridcode = 0 (represents areas viewer cannot see)
    with arcpy.da.UpdateCursor("Poly_outView", ['*']) as cursor:
        for row in cursor:
            if row[3] == 0:
                cursor.deleteRow()
    print("Saving & Updating: Poly_outView")

    ###################### create a min bounding circle on vectorized viewshed raster ######################
    # creating minimum bounding circle around viewshds
    arcpy.management.MinimumBoundingGeometry("Poly_outView", 
                                         "B_outView", 
                                         "CIRCLE", 
                                         "ALL")
    print("Saving B_outView (min bounding geom)")

    ###################### Generate points on perimeter of circle (min bounding region) ######################
    # creating pts on minimum bounding circle
    arcpy.management.GeneratePointsAlongLines("B_outView", 
                                          "Pts_View", 
                                          "PERCENTAGE",  
                                          Percentage=0.2, 
                                          Include_End_Points='END_POINTS')

    print("Saving Pts_View (pts on min bounding geom)")
    # takes apporx 2-3 mins to execute

    # Running the function that'll create polylines for each image and corresponding viewshed points
    print("Creating: View_lines (501 poly_lines from image center to pts on min bound geom)")
    creating_polylines("View_lines", image_pt, "Pts_View", 0.2)
    # this step takes forever 10+ mins

    ###################### Use vectorized viewshed raster to mask DEM ###################### 
    # declaring the spatial extent of the DEM1 (used to help clip the DEM to the viewshed output)
    extent = DEM.extent
    xmin = extent.XMin
    ymin = extent.YMin
    xmax = extent.XMax
    ymax = extent.YMax
    spatial_extent = str(xmin) + " " + str(ymin) +  " " + str(xmax) +  " " +str(ymax)

    # creating a clipped extent of DEM for each image
    print("Clipping: Clipped_DEM_Vwshd")
    arcpy.management.Clip(DEM, 
                          spatial_extent,
                          "Clipped_DEM_Vwshd", 
                          "Poly_outView", 
                          "-999999", 
                          "ClippingGeometry", 
                          "NO_MAINTAIN_EXTENT")

    ################### Extract the Elevation values from the clipped viewshed DEM with the Polylines ################### 
    # convert elevation datatype from decimal to integer (looses some data)
    print("Saving Int_clip_DEM_Vwshd")
    out_raster = arcpy.ia.Int("Clipped_DEM_Vwshd"); out_raster.save("Int_clip_DEM_Vwshd")

    # convert raster to polygons (s.t. each raster pixel is a polygon)
    arcpy.conversion.RasterToPolygon("Int_clip_DEM_Vwshd", 
                                     "Poly_Int_Clip_DEM", 
                                     "NO_SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

    # spatial join between pixels (as polygons) and polylines : whilst saving only max elevations for each polyline
    # this join returns the max 'gridcode' ('elevation value') that intersects with a line!!
    arcpy.analysis.SpatialJoin("View_lines", 
                               "Poly_Int_Clip_DEM", 
                               "lines_max_elev", 
                               "JOIN_ONE_TO_ONE", 
                               "KEEP_ALL", 
                               'Degree "Degree" true true false 19 Double 0 0,First,#,view1_lines_b,Degree,-1,-1;'\
                               'gridcode "gridcode" true true false 4 Long 0 0,Max,#,Poly_Int_Clip_DEM,gridcode,-1,-1', 
                               "INTERSECT", 
                               None, 
                               '')
    print("Saving Max Elevation value")
    
    ############################## Spatial Join between lines and DEM pixels Keeping Elevation Geom ###############################
    # creating a Spatial Join where we keep the elevation geom: 
    arcpy.analysis.SpatialJoin("Poly_Int_Clip_DEM", 
                               "lines_max_elev", 
                               "Poly_Int_Clip_DEM_SpJn", 
                               "JOIN_ONE_TO_MANY", 
                               "KEEP_COMMON", 
                               'gridcode "gridcode" true true false 4 Long 0 0,First,#,Poly_Int_Clip_DEM,gridcode,-1,-1;'\
                               'Degree "Degree" true true false 19 Double 0 0,First,#,view1_lines_b,Degree,-1,-1;'\
                               'Line_ID "Line_ID" true true false 19 Double 0 0,First,#,view1_lines_b,Line_ID,-1,-1', 
                               "INTERSECT", 
                               None, 
                               '')
    # this takes a WHILE (at least 1 minutes)
    print("Finished Spatial Join")

    # indentifying MAX elevation for each polyline
    max_dict1 = Identifying_Max_elevations_from_Poly_Pixels("lines_max_elev", "Poly_Int_Clip_DEM_SpJn")
    # again, this can take a few minutes ~2 minutes

    # deleting poylgons from the geometry pixels that have not been identified as the pixels with max elevation for each polyline
    Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict1, "Poly_Int_Clip_DEM_SpJn")
    print("deleting Non-max pixels")

    # clipping lines based on their intersection with Max pixels for each line
    Splitting_polylines_by_Max_Pixel_geom("Poly_Int_Clip_DEM_SpJn", 
                                          "lines_poly_Intersect", 
                                          "lines_max_elev", 
                                          "lines_Split_view", 
                                          max_dict1)
    print("splitting lines")   
    ################# Calculate Geometry (line Length) #################
    sr = arcpy.SpatialReference(4269)
    arcpy.management.CalculateGeometryAttributes("lines_Split_view", 
                                                 "LENGTH LENGTH_GEODESIC", 
                                                 "KILOMETERS", 
                                                 '', 
                                                 sr, 
                                                 "SAME_AS_INPUT")
    print("calculating distance (a.k.a line length)")
    # reformating table bfore export
    arcpy.management.DeleteField("lines_Split_view", "Degree;gridcode;LENGTH","KEEP_FIELDS")
    arcpy.management.AlterField("lines_Split_view", "gridcode", "Elevation", "Elevation")
    arcpy.management.AlterField("lines_Split_view", "LENGTH", "Distance", "Distance")

    # exporting Distances associated with each of the polyline as a csv...
    arcpy.conversion.ExportTable("lines_Split_view", 
                                 out_path + "\\" + out_file_name + ".csv", 
                                 '', 
                                 "NOT_USE_ALIAS"
                                )
    print("saving to a csv")

In [7]:
# takes approximatley 13-30 mins to run for each image 
# This cell takes a long time to run.... 
# I honestly don't know why since most o fthe code is the same but reformatted from earlier versions
# Export_CSV_for_360_Ridgeline(List_shp[0], Raster(DEM_Filepath), "lines_Split_view1", out_path)
# Export_CSV_for_360_Ridgeline(List_shp[1], Raster(DEM_Filepath), "lines_Split_view2", out_path)
# Export_CSV_for_360_Ridgeline(List_shp[2], Raster(DEM_Filepath), "lines_Split_view3", out_path)
Export_CSV_for_360_Ridgeline(List_shp[3], Raster(DEM_Filepath), "lines_Split_view4", out_path)
# Export_CSV_for_360_Ridgeline(List_shp[4], Raster(DEM_Filepath), "lines_Split_view5", out_path)
# Export_CSV_for_360_Ridgeline(List_shp[5], Raster(DEM_Filepath), "lines_Split_view6", out_path)

saving outViewshed
Saving & Updating: Poly_outView
Saving B_outView (min bounding geom)
Saving Pts_View (pts on min bounding geom)
Creating: View_lines (501 poly_lines from image center to pts on min bound geom)
Clipping: Clipped_DEM_Vwshd
Saving Int_clip_DEM_Vwshd
Saving Max Elevation value
Finished Spatial Join
Done Creating Max Dict. Iterated 13922790 times (lines x pixels in each viewshed)
deleting Non-max pixels
Number of matches found: 963
splitting lines
calculating distance (a.k.a line length)
saving to a csv
